# My Scratchpad for Zillow Clustering Project

In [9]:
#imports
import pandas as pd
import numpy as np
import acquire
import prepare
import explore
from env import host, user, password

## Acquire Zillow Data

In [10]:
df = acquire.get_zillow_data(cached=True)
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,1087254,10711855,NaN,NaN,NaN,2.0,3.0,NaN,8.0,2.0,...,6.037113e+13,-0.007357,2017-07-07,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
1,1072280,10711877,1.0,NaN,NaN,2.0,4.0,NaN,8.0,2.0,...,6.037113e+13,0.021066,2017-08-29,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN
2,1340933,10711888,1.0,NaN,NaN,2.0,4.0,NaN,8.0,2.0,...,6.037113e+13,0.077174,2017-04-04,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN
3,1878109,10711910,NaN,NaN,NaN,2.0,3.0,NaN,8.0,2.0,...,6.037113e+13,-0.041238,2017-03-17,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,2190858,10711923,NaN,NaN,NaN,2.0,4.0,NaN,8.0,2.0,...,6.037113e+13,-0.009496,2017-03-24,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN


In [11]:
df.shape

(77575, 68)

In [12]:
df = prepare.wrangle_zillow()

In [13]:
df.shape

(69920, 25)

In [14]:
df.isnull().sum()

bathroomcnt                     0
bedroomcnt                      0
buildingqualitytypeid           0
calculatedfinishedsquarefeet    0
fips                            0
latitude                        0
longitude                       0
lotsizesquarefeet               0
rawcensustractandblock          0
regionidcity                    0
regionidcounty                  0
regionidzip                     0
roomcnt                         0
unitcnt                         0
yearbuilt                       0
structuretaxvaluedollarcnt      0
taxvaluedollarcnt               0
assessmentyear                  0
landtaxvaluedollarcnt           0
taxamount                       0
logerror                        0
transactiondate                 0
heatingorsystemdesc             0
county                          0
logerror_class                  0
dtype: int64

In [15]:
df.head(1)

,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,rawcensustractandblock,regionidcity,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,logerror,transactiondate,heatingorsystemdesc,county,logerror_class
parcelid,,,,,,,,,,,,,,,,,,,,,
10711855,2.0,3.0,8.0,2107.0,6037.0,34.222559,-118.617387,9158.0,6.037113e+07,12447.0,...,249655.0,624139.0,2016.0,374484.0,7659.36,-0.007357,2017-07-07,Central,Los_Angeles,q2


In [17]:
df = explore.get_county_dummies(df)
df.head()

,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,rawcensustractandblock,regionidcity,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,logerror,transactiondate,heatingorsystemdesc,logerror_class,Orange,Ventura
parcelid,,,,,,,,,,,,,,,,,,,,,
10711855,2.0,3.0,8.0,2107.0,6037.0,34.222559,-118.617387,9158.0,6.037113e+07,12447.0,...,624139.0,2016.0,374484.0,7659.36,-0.007357,2017-07-07,Central,q2,0,0
10711877,2.0,4.0,8.0,1882.0,6037.0,34.220261,-118.616409,9035.0,6.037113e+07,12447.0,...,660000.0,2016.0,407000.0,8123.91,0.021066,2017-08-29,Central,q3,0,0
10711888,2.0,4.0,8.0,1882.0,6037.0,34.222491,-118.616854,9800.0,6.037113e+07,12447.0,...,542923.0,2016.0,285332.0,6673.24,0.077174,2017-04-04,Central,q4,0,0
10711910,2.0,3.0,8.0,1477.0,6037.0,34.221864,-118.615739,11285.0,6.037113e+07,12447.0,...,78031.0,2016.0,20063.0,1116.46,-0.041238,2017-03-17,Central,q1,0,0
10711923,2.0,4.0,8.0,1918.0,6037.0,34.220619,-118.615253,11239.0,6.037113e+07,12447.0,...,415459.0,2016.0,247590.0,5239.85,-0.009496,2017-03-24,Central,q2,0,0
